# 任务3：LSTM网络实现RNN写作生成

**任务要求：利用LSTM网络实现RNN的莎士比亚风格文本写作生成。**  
**数据: shakespeare.txt**

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import unidecode
import string
import random
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
use_gpu = True if torch.cuda.is_available() else False
print('Use GPU:', use_gpu)

## 1. 加载文本数据

加载训练文本数据及字符集

In [ ]:
all_characters = string.printable
n_characters = len(all_characters)
print('characters num:', n_characters)
print(all_characters)

file = unidecode.unidecode(open('./data/shakespeare.txt').read())
file_len = len(file)
print('file length:', file_len)

### 随机文本块

In [ ]:
chunk_len = 500

def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index: end_index]

print(random_chunk())

### 随机数据-标签对

In [ ]:
def char_tensor(string):
    n = len(string)
    t = torch.zeros(n).long()
    for i in range(n):
        t[i] = all_characters.index(string[i])
    return Variable(t)

print(char_tensor('(abcdef,ABCDEF).'))

In [ ]:
def random_training_set():
    chunk = random_chunk()
    inputs = char_tensor(chunk[:-1])
    targets = char_tensor(chunk[1:])
    return inputs, targets

random_training_set()

## 2. RNN模型

利用PyTorch构建RNN模型类，内部包括Embedding层、LSTM、FC层

In [7]:
''' 请在下方编写RNN模型类 '''
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()

        
        
        
        
        
        
        
        
    def forward(self, inputs, hidden):
        outputs = None
        
        
        
        return outputs, hidden
    
    def init_hidden(self):
        h0, c0 = None, None
        return h0, c0

## 3. 训练函数

In [ ]:
def train(inputs, target):
    h, c = rnn.init_hidden()
    if use_gpu:
        inputs = inputs.cuda()
        target = target.cuda()
        h, c = h.cuda(), c.cuda()
    hidden = h, c
    
    loss = 0
    for i in range(len(inputs)):
        ''' 请在下方编写rnn模型输出，及loss累加代码'''

    
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.data.item()/len(inputs)

## 4. 文本生成测试函数

In [ ]:
def test(prime_str='A', predict_len=100, temperature=0.8):
    h, c = rnn.init_hidden()
    prime_input = char_tensor(prime_str)
    if use_gpu:
        h, c = h.cuda(), c.cuda()
        prime_input = prime_input.cuda()
    predicted = prime_str
    
    for i in range(len(prime_str)-1):
        _, (h, c) = rnn(prime_input[i], (h, c))
    inputs = prime_input[-1]

    for p in range(predict_len):
        if use_gpu:
            inputs = inputs.cuda()
        output, (h, c) = rnn(inputs, (h, c))
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]

        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inputs = char_tensor(predicted_char)
    
    return predicted

## 5. 模型训练

#### 超参数

In [ ]:
''' 请自行设定相关参数 '''
epoch_num = 2000  # 1000~10000
print_every = 100
plot_every = 10
hidden_size = 100
num_layers = 1
lr = 0.01

#### 模型、优化器、损失函数

In [ ]:
# RNN模型
rnn = RNN(n_characters, hidden_size, n_characters, num_layers)
if use_gpu:
    rnn = rnn.cuda()
print(rnn)

In [ ]:
# 优化器
''' 请在下方编写Adam优化器定义的代码 '''
optimizer = None
print(optimizer)

# 损失函数
''' 请在下方编写交叉熵损失函数定义的代码 '''
criterion = None
print(criterion)

#### 模型训练

In [ ]:
time_start = time.time()
evaluate = True  # 可设置为False，训练过程中不评估
loss_list = []
loss_avg = 0

for epoch in range(1, epoch_num+1):
    inputs, targets = random_training_set()
    loss = train(inputs, targets)
    loss_avg += loss
    
    if epoch % print_every == 0:
        print('[Time: %ds  Epoch: %d (%d%%)  Loss: %.4f]'%(int(time.time()-time_start), epoch, epoch/epoch_num*100, loss))
        if evaluate:
            pred_text = test('Th', 200)
            print(pred_text, '\n')
        
    if epoch % plot_every == 0:
        loss_list.append(loss_avg/plot_every)
        loss_avg = 0

## 6. 训练Loss曲线

绘制训练过程中的Loss曲线

In [ ]:
plt.figure(figsize=(12, 7))
plt.plot(loss_list)
plt.title('Train Loss', fontsize=18)
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid()

## 7. 测试

自定义初始字符和文本长度以生成文本

In [ ]:
test_text = test('Th', 1000, 0.8)
print(test_text)

In [ ]:
test_text = test('Wh', 1000, 0.8)
print(test_text)